# Initial Setup
All new data is stored into two new collections called climate_partb and hotspot_partb which both follow the exact same data models as the previous questions. This is done simply to keep things organised

In [14]:
import pymongo
from pymongo import MongoClient

client = MongoClient()
db = client.fit3182_assignment_db
collection_climate_partb = db.climate_partb
collection_hotspot_partb = db.hotspot_partb

First, import our database client, spark session entry point, and some basic spark sql functions for data transformation.

In [15]:
from pymongo import MongoClient
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, element_at, when

from json import dumps
from json import loads
import pygeohash as pgh
import numpy as np
import datetime as dt
import random

Set a global variable to store topic name.

In [16]:
topic_name = 'assignment'

Initialize our spark session with `#threads = #logicalCPU` and the given application name.

In [17]:
spark = (
    SparkSession.builder
    .master('local[*]')
    .appName('[Demo] Spark Streaming from Kafka into MongoDB')
    .getOrCreate()
)

Create a streaming dataframe with options providing the bootstrap server(s) and topic name.

In [18]:
topic_stream_df = (
    spark.readStream.format('kafka')
    .option('kafka.bootstrap.servers', 'localhost:9092')
    .option('subscribe', topic_name)
    .load()
)

We can print the schema for this dataframe to see what columns we have to work with.

In [19]:
topic_stream_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [20]:
#we make changes to our topic_stream in the process_batches function
output_stream_df = topic_stream_df

Define a utility class to process the rows of our streaming dataframe.

In [21]:
class DbWriter:
    # called at the start of processing each partition in each output micro-batch
    def open(self, partition_id, epoch_id):
        self.mongo_client = MongoClient(
            host='localhost',
            port=27017
        )
        self.db = self.mongo_client['fit3182_db']
        return True
    
    # called once per row of the result dataframe
    # the current code DOES NOT handle duplicate processing
    #   e.g., query fails and restarts just before current micro-batch was fully inserted
    def process(self, row):
        self.db[topic_name].insert_one(row.asDict())
    
    # called once all rows have been processed (possibly with error)
    def close(self, err):
        self.mongo_client.close()

Now, define our stream writer for the MongoDB database sink.

# Process Batch
Here we process the incoming data by checking whether there is a fire and then storing the information in the relevant collections. There are print statements that print the incoming raw data and then the collection in which the data was stored and whether the was no fire, natural fire or other fire

In [22]:
def process_batch(batch_df, batch_id):
    raw_data = batch_df.collect()
    
    #store each row in our batch to rows_in_batch
    rows_in_batch = []
    
    #get each row in batch and append all rows to rows_in_batch
    for row in raw_data:
        
        #get the value row as a string
        x = row.value
        x = x.decode()
        
        #use the json loads function to convert our string into an array containing our producerid and our 
        #json object
        x = loads(x)
        
        rows_in_batch.append(x)
        
    #DELETE LATER
    print("rows_in_batch")
    print(rows_in_batch)
    
    
    #check if there is climate data, check is True if there is climate_data
    check, climate_data = check_climate_data_present(rows_in_batch)
    
    if check:
        
        #calculate geohash for climate_data
        climate_geohash = get_geohash(climate_data, precision=3)
        
        #get all hotspot data with same geohash as climate_geohash
        hotspot_data = group_by_location(rows_in_batch, climate_geohash)
        
        #if no hotspot data, we just append the climate_data at the end
        
        #else if there is more than one hotspot (the precision 3 geohash is the same)
        if len(hotspot_data) > 1:
            
            #group hotspot_data by geohash of precision 5
            hotspot_data = group_by_location_precision5(hotspot_data)
            
            #merge the hotspot_data groups
            hotspot_data = merge_hotspot(hotspot_data)
            
        
        #insert data into collection_hotspot_partb for every hotspot in hotspot_data
        for i in range(len(hotspot_data)):
            
            #check if fire was natural
            if climate_data[1]["air_temperature_celcius"] > 20 and climate_data[1]["GHI_w/m2"] > 180:
                
                #create json object to insert into collection
                #date and datetime is used from our climate_data
                #a random number of hours between 0 and 23 is added to datetime
                random_hours = random.randrange(0, 23)
                datetime_random_hours = dt.datetime.strptime(climate_data[1]["date"],"%d/%m/%Y")
                datetime_random_hours += dt.timedelta(hours=random_hours)
                
                newEntry = {"latitude": hotspot_data[i][1]["latitude"],
                            "longitude": hotspot_data[i][1]["longitude"],
                            "datetime": datetime_random_hours,
                            "confidence": int(hotspot_data[i][1]["confidence"]),
                            "date": dt.datetime.strptime(climate_data[1]["date"],"%d/%m/%Y"),
                            "surface_temperature_celcius": int(hotspot_data[i][1]["surface_temperature_celcius"]),
                            "cause_of_fire" : "natural"}
                
                #insert into collection_hotspot_partb
                collection_hotspot_partb.insert_one(newEntry)
                
                #DELETE LATER
                print("natural fire")
                print("natural fire")
                print("natural fire")
                print("natural fire")
                print(newEntry)
                print("\n")
        
            
            #else if fire cause was other
            else:
                
                #create json object to insert into collection
                #date and datetime is used from our climate_data
                #a random number of hours between 0 and 23 is added to datetime
                random_hours = random.randrange(0, 23)
                datetime_random_hours = dt.datetime.strptime(climate_data[1]["date"],"%d/%m/%Y")
                datetime_random_hours += dt.timedelta(hours=random_hours)
                
                newEntry = {"latitude": hotspot_data[i][1]["latitude"],
                            "longitude": hotspot_data[i][1]["longitude"],
                            "datetime": datetime_random_hours,
                            "confidence": int(hotspot_data[i][1]["confidence"]),
                            "date": dt.datetime.strptime(climate_data[1]["date"],"%d/%m/%Y"),
                            "surface_temperature_celcius": int(hotspot_data[i][1]["surface_temperature_celcius"]),
                            "cause_of_fire" : "other"}
                
                #insert into collection_hotspot_partb
                collection_hotspot_partb.insert_one(newEntry)
                
                #DELETE LATER
                print("other fire")
                print("other fire")
                print("other fire")
                print("other fire")
                print(newEntry)
                print("\n")
                
    
        #insert climate data into collection_climate_partb
        climate_data[1]["date"] = dt.datetime.strptime(climate_data[1]["date"],"%d/%m/%Y")
        newEntry = climate_data[1]
        collection_climate_partb.insert_one(newEntry)

        #DELETE LATER
        print("no fire")
        print(newEntry)
        print("\n")
            
    
    

# Helper Functions
Here we have all the helper functions that were used to simplify our code in the process_batch() function. These helper functions have their use commented on top of them

In [23]:
#this function returns the geohash value for a given row of data
def get_geohash(row_data, precision=3):
    return pgh.encode(row_data[1]["latitude"], row_data[1]["longitude"], precision)
    

#this function takes in a geohash for the climate data and returns all hotspot data with the same geohash of 
#precision 3
def group_by_location(rows_in_batch, climate_geohash):    
    res = []
    
    for i in range(len(rows_in_batch)):
        if rows_in_batch[i][0] != 'producer1' and get_geohash(rows_in_batch[i], precision=3) == climate_geohash:
            res.append(rows_in_batch[i])
            
        return res

#this function returns True and the row itself if rows_in_batch contains data from producer1 else False
def check_climate_data_present(rows_in_batch):
    for i in range(len(rows_in_batch)):
        if rows_in_batch[i][0] == 'producer1':
            return True, rows_in_batch[i]
    
    return False, False

#this function groups together the hotspot data based on their geohash of precision5
def group_by_location_precision5(hotspot_data):
    
    #loop through hotspot_data and add any new geohash to keys array and group together data with same geohash valu
    keys = []
    groups = []
    
    for i in range(len(hotspot_data)):
        geohash = get_geohash(hotspot_data[i], precision=5)
        
        if geohash in keys:
            index = keys.index(geohash)
            
            groups[index].append(hotspot_data[i])
            
        else:
            keys.append(geohash)
            groups.append([hotspot_data[i]])
            
    return groups


#this function merges hotspot_data groups that were previously grouped together
def merge_hotspot(hotspot_data_groups):
    res = []
    
    for hotspot_data in hotspot_data_groups:
        
        #if more than one hotspot in the group
        if len(hotspot_data) > 1:
    
            #average all latitudes
            avg_latitude = []
            for i in range(len(hotspot_data)):
                avg_latitude.append(hotspot_data[i][1]["latitude"])
            avg_latitude = sum(avg_latitude) / len(avg_latitude)

            #average all longitudes
            avg_longitude = []
            for i in range(len(hotspot_data)):
                avg_longitude.append(hotspot_data[i][1]["longitude"])
            avg_longitude = sum(avg_longitude) / len(avg_longitude)

            #average all confidences
            avg_confidence = []
            for i in range(len(hotspot_data)):
                avg_confidence.append(hotspot_data[i][1]["confidence"])
            avg_confidence = sum(avg_confidence) / len(avg_confidence)

            #average all surface tempteratures
            avg_surface_temperature_celcius = []
            for i in range(len(hotspot_data)):
                avg_surface_temperature_celcius.append(hotspot_data[i][1]["surface_temperature_celcius"])
            avg_surface_temperature_celcius = sum(avg_surface_temperature_celcius) / len(avg_surface_temperature_celcius)

            avg_data = {"latitude": avg_latitude,
                        "longitude": avg_longitude,
                        "datetime": "no date",
                        "confidence": avg_confidence,
                        "date": "no date",
                        "surface_temperature_celcius": avg_surface_temperature_celcius}
            
            #since we cant merge the name of producer2 and producer3, we just put it as producer2
            res.append(["producer2", avg_data])
            
        else:
            #single hotspot in group
            avg_data = hotspot_data[0]
            
            #since we cant merge the name of producer2 and producer3, we just put it as producer2
            res.append(hotspot_data[0])
    
    return res

In [24]:
db_writer = (
    output_stream_df
    .writeStream
    .outputMode('append')
    .trigger(processingTime="10 seconds")
    .foreachBatch(process_batch)
)

The variable `writer` points to the stream writer we wish to use.

In [25]:
# change this variable to either db_writer or console_logger to get output to the desired sink
writer = db_writer

Now we can start our streaming query. The workflow is straightforward.
1. Call the `StreamWriter` object's `start` method to begin execution of the query.
1. The `query` variable allows us to manage the running query. This is a simple demo, so we use `awaitTermination` to block our driver program until the streaming query is stopped due to failure or user-interrupts.

You can press CTRL-C in the notebook to stop the running query anytime.

In [26]:
try:
    query = writer.start()
    query.awaitTermination()
except KeyboardInterrupt:
    print('Interrupted by CTRL-C. Stopped query')
except StreamingQueryException as exc:
    print(exc)
finally:
    query.stop()

rows_in_batch
[]
rows_in_batch
[['producer2', {'latitude': -37.7021, 'longitude': 145.5536, 'datetime': 'no date', 'confidence': 88, 'date': 'no date', 'surface_temperature_celcius': 42}], ['producer3', {'latitude': -36.5511, 'longitude': 146.7819, 'datetime': 'no date', 'confidence': 56, 'date': 'no date', 'surface_temperature_celcius': 34}], ['producer2', {'latitude': -37.5043, 'longitude': 146.3299, 'datetime': 'no date', 'confidence': 100, 'date': 'no date', 'surface_temperature_celcius': 93}], ['producer3', {'latitude': -37.3165, 'longitude': 147.4932, 'datetime': 'no date', 'confidence': 93, 'date': 'no date', 'surface_temperature_celcius': 72}], ['producer2', {'latitude': -37.6674, 'longitude': 142.2687, 'datetime': 'no date', 'confidence': 65, 'date': 'no date', 'surface_temperature_celcius': 42}], ['producer3', {'latitude': -36.4074, 'longitude': 145.9085, 'datetime': 'no date', 'confidence': 74, 'date': 'no date', 'surface_temperature_celcius': 48}], ['producer2', {'latitude'

rows_in_batch
[['producer2', {'latitude': -36.2851, 'longitude': 145.8, 'datetime': 'no date', 'confidence': 88, 'date': 'no date', 'surface_temperature_celcius': 64}], ['producer3', {'latitude': -34.2594, 'longitude': 141.6139, 'datetime': 'no date', 'confidence': 81, 'date': 'no date', 'surface_temperature_celcius': 54}], ['producer2', {'latitude': -36.9959, 'longitude': 148.2118, 'datetime': 'no date', 'confidence': 95, 'date': 'no date', 'surface_temperature_celcius': 75}], ['producer3', {'latitude': -37.0623, 'longitude': 142.8211, 'datetime': 'no date', 'confidence': 80, 'date': 'no date', 'surface_temperature_celcius': 53}], ['producer2', {'latitude': -36.3492, 'longitude': 140.9727, 'datetime': 'no date', 'confidence': 88, 'date': 'no date', 'surface_temperature_celcius': 63}], ['producer3', {'latitude': -37.1999, 'longitude': 143.6425, 'datetime': 'no date', 'confidence': 62, 'date': 'no date', 'surface_temperature_celcius': 41}], ['producer2', {'latitude': -35.9438, 'longitud

rows_in_batch
[['producer2', {'latitude': -36.7239, 'longitude': 141.0154, 'datetime': 'no date', 'confidence': 70, 'date': 'no date', 'surface_temperature_celcius': 45}], ['producer3', {'latitude': -36.5069, 'longitude': 144.7323, 'datetime': 'no date', 'confidence': 81, 'date': 'no date', 'surface_temperature_celcius': 54}], ['producer2', {'latitude': -37.4591, 'longitude': 144.9492, 'datetime': 'no date', 'confidence': 100, 'date': 'no date', 'surface_temperature_celcius': 89}], ['producer3', {'latitude': -36.9664, 'longitude': 142.1561, 'datetime': 'no date', 'confidence': 81, 'date': 'no date', 'surface_temperature_celcius': 55}], ['producer2', {'latitude': -35.2378, 'longitude': 142.9864, 'datetime': 'no date', 'confidence': 100, 'date': 'no date', 'surface_temperature_celcius': 94}], ['producer3', {'latitude': -37.623000000000005, 'longitude': 149.284, 'datetime': 'no date', 'confidence': 84, 'date': 'no date', 'surface_temperature_celcius': 61}], ['producer2', {'latitude': -37.

rows_in_batch
[['producer2', {'latitude': -37.1945, 'longitude': 148.8074, 'datetime': 'no date', 'confidence': 83, 'date': 'no date', 'surface_temperature_celcius': 57}], ['producer3', {'latitude': -37.4621, 'longitude': 143.0869, 'datetime': 'no date', 'confidence': 75, 'date': 'no date', 'surface_temperature_celcius': 48}], ['producer2', {'latitude': -36.5023, 'longitude': 143.6038, 'datetime': 'no date', 'confidence': 85, 'date': 'no date', 'surface_temperature_celcius': 66}], ['producer3', {'latitude': -36.1198, 'longitude': 145.6369, 'datetime': 'no date', 'confidence': 79, 'date': 'no date', 'surface_temperature_celcius': 52}], ['producer2', {'latitude': -37.7856, 'longitude': 143.4095, 'datetime': 'no date', 'confidence': 67, 'date': 'no date', 'surface_temperature_celcius': 43}], ['producer3', {'latitude': -34.2782, 'longitude': 142.2253, 'datetime': 'no date', 'confidence': 81, 'date': 'no date', 'surface_temperature_celcius': 54}], ['producer2', {'latitude': -36.7084, 'longi

rows_in_batch
[['producer2', {'latitude': -36.7223, 'longitude': 143.9213, 'datetime': 'no date', 'confidence': 73, 'date': 'no date', 'surface_temperature_celcius': 47}], ['producer3', {'latitude': -36.1941, 'longitude': 143.1604, 'datetime': 'no date', 'confidence': 93, 'date': 'no date', 'surface_temperature_celcius': 72}], ['producer2', {'latitude': -36.2403, 'longitude': 141.4047, 'datetime': 'no date', 'confidence': 64, 'date': 'no date', 'surface_temperature_celcius': 42}], ['producer3', {'latitude': -36.7842, 'longitude': 146.9658, 'datetime': 'no date', 'confidence': 53, 'date': 'no date', 'surface_temperature_celcius': 39}], ['producer2', {'latitude': -36.8293, 'longitude': 141.9382, 'datetime': 'no date', 'confidence': 79, 'date': 'no date', 'surface_temperature_celcius': 52}], ['producer3', {'latitude': -36.4309, 'longitude': 141.1436, 'datetime': 'no date', 'confidence': 80, 'date': 'no date', 'surface_temperature_celcius': 54}], ['producer2', {'latitude': -36.9576, 'longi

...

rows_in_batch
[['producer3', {'latitude': -37.803000000000004, 'longitude': 142.3151, 'datetime': 'no date', 'confidence': 83, 'date': 'no date', 'surface_temperature_celcius': 78}], ['producer2', {'latitude': -36.3911, 'longitude': 141.1262, 'datetime': 'no date', 'confidence': 73, 'date': 'no date', 'surface_temperature_celcius': 47}], ['producer3', {'latitude': -36.3405, 'longitude': 145.4751, 'datetime': 'no date', 'confidence': 67, 'date': 'no date', 'surface_temperature_celcius': 43}], ['producer2', {'latitude': -36.4437, 'longitude': 141.5916, 'datetime': 'no date', 'confidence': 58, 'date': 'no date', 'surface_temperature_celcius': 45}], ['producer3', {'latitude': -36.3014, 'longitude': 143.203, 'datetime': 'no date', 'confidence': 95, 'date': 'no date', 'surface_temperature_celcius': 76}], ['producer2', {'latitude': -36.6595, 'longitude': 143.8887, 'datetime': 'no date', 'confidence': 72, 'date': 'no date', 'surface_temperature_celcius': 46}], ['producer3', {'latitude': -36.66

rows_in_batch
[['producer3', {'latitude': -37.399, 'longitude': 148.08100000000002, 'datetime': 'no date', 'confidence': 100, 'date': 'no date', 'surface_temperature_celcius': 84}], ['producer2', {'latitude': -36.2549, 'longitude': 141.9908, 'datetime': 'no date', 'confidence': 69, 'date': 'no date', 'surface_temperature_celcius': 44}], ['producer3', {'latitude': -37.665, 'longitude': 142.1339, 'datetime': 'no date', 'confidence': 74, 'date': 'no date', 'surface_temperature_celcius': 48}], ['producer2', {'latitude': -37.5422, 'longitude': 143.2276, 'datetime': 'no date', 'confidence': 75, 'date': 'no date', 'surface_temperature_celcius': 49}], ['producer3', {'latitude': -36.3585, 'longitude': 141.5686, 'datetime': 'no date', 'confidence': 72, 'date': 'no date', 'surface_temperature_celcius': 47}], ['producer2', {'latitude': -37.59, 'longitude': 149.31, 'datetime': 'no date', 'confidence': 51, 'date': 'no date', 'surface_temperature_celcius': 42}], ['producer3', {'latitude': -37.0653, '

rows_in_batch
[['producer2', {'latitude': -36.3126, 'longitude': 145.8104, 'datetime': 'no date', 'confidence': 76, 'date': 'no date', 'surface_temperature_celcius': 63}], ['producer3', {'latitude': -37.3601, 'longitude': 145.8519, 'datetime': 'no date', 'confidence': 82, 'date': 'no date', 'surface_temperature_celcius': 62}], ['producer2', {'latitude': -36.8237, 'longitude': 141.4881, 'datetime': 'no date', 'confidence': 84, 'date': 'no date', 'surface_temperature_celcius': 58}], ['producer3', {'latitude': -38.0219, 'longitude': 141.2238, 'datetime': 'no date', 'confidence': 76, 'date': 'no date', 'surface_temperature_celcius': 62}], ['producer2', {'latitude': -35.2464, 'longitude': 141.1143, 'datetime': 'no date', 'confidence': 98, 'date': 'no date', 'surface_temperature_celcius': 45}], ['producer3', {'latitude': -38.036, 'longitude': 143.9012, 'datetime': 'no date', 'confidence': 67, 'date': 'no date', 'surface_temperature_celcius': 45}], ['producer2', {'latitude': -36.2726, 'longit

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/student/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/student/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


Interrupted by CTRL-C. Stopped query
